<a href="https://colab.research.google.com/github/rastringer/promptcraft/blob/main/chain_of_thought.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Chain-of-thought-prompting

In this notebook, we will explore how we can ask the chat model to show us its conclusions in a multi-step process. Such operations would typically be masked from the user and serve to help developers test the chat application.

In [ ]:
# !pip install "shapely<2.0.0"
# !pip install google-cloud-aiplatform

If you're on Colab, run the following cell to authenticate

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

In [ ]:
from google.cloud import aiplatform as vertexai

In [ ]:
import vertexai
from vertexai.preview.language_models import ChatModel, InputOutputTextPair

# Replace the project and location placeholder values below
vertexai.init(project="<your-project-id>", location="<location>")
chat_model = ChatModel.from_pretrained("chat-bison@001")
parameters = {
    "temperature": 0.2,
    "max_output_tokens": 256,
    "top_p": 0.8,
    "top_k": 40
}

In [ ]:
products = """
name: Caffeino Classic
category: Espresso Machines
brand: EliteBrew
model_number: EB-1001
warranty: 2 years
rating: 4.6/5 stars
features:
  15-bar pump for authentic espresso extraction.
  Milk frother for creating creamy cappuccinos and lattes.
  Removable water reservoir for easy refilling.
description: The Caffeino Classic by EliteBrew is a powerful espresso machine that delivers rich and flavorful shots of espresso with the convenience of a built-in milk frother, perfect for indulging in your favorite cafe-style beverages at home.
price: £179.99

name: BeanPresso
category: Single Serve Coffee Makers
brand: FreshBrew
model_number: FB-500
warranty: 1 year
rating: 4.3/5 stars
features:
  Compact design ideal for small spaces or travel.
  Compatible with various coffee pods for quick and easy brewing.
  Auto-off feature for energy efficiency and safety.
description: The BeanPresso by FreshBrew is a compact single-serve coffee maker that allows you to enjoy a fresh cup of coffee effortlessly using your favorite coffee pods, making it the perfect companion for those with limited space or always on the go.
price: £49.99

name: BrewBlend Pro
category: Drip Coffee Makers
brand: MasterRoast
model_number: MR-800
warranty: 3 years
rating: 4.7/5 stars
features:
  Adjustable brew strength for customized coffee flavor.
  Large LCD display with programmable timer for convenient brewing.
  Anti-drip system to prevent messes on the warming plate.
description: The BrewBlend Pro by MasterRoast offers a superior brewing experience with adjustable brew strength, programmable timer, and anti-drip system, ensuring a perfectly brewed cup of coffee every time, making mornings more enjoyable.
price: £89.99

name: SteamGenie
category: Stovetop Coffee Makers
brand: KitchenWiz
model_number: KW-200
warranty: 2 years
rating: 4.4/5 stars
features:
  Classic Italian stovetop design for rich and aromatic coffee.
  Durable stainless steel construction for long-lasting performance.
  Available in multiple sizes to suit different brewing needs.
description: The SteamGenie by KitchenWiz is a traditional stovetop coffee maker that harnesses the essence of Italian coffee culture, crafted with durable stainless steel and delivering a rich, authentic coffee experience with every brew.
price: £39.99

name: AeroBlend Max
category: Coffee and Espresso Combo Machines
brand: AeroGen
model_number: AG-1200
warranty: 2 years
rating: 4.9/5 stars
features:
  Dual-functionality for brewing coffee and espresso.
  Built-in burr grinder for fresh coffee grounds.
  Adjustable temperature and brew strength settings for personalized beverages.
description: The AeroBlend Max by AeroGen is a versatile coffee and espresso combo machine that combines the convenience of brewing both coffee and espresso with a built-in grinder,
allowing you to enjoy the perfect cup of your preferred caffeinated delight with ease.
price: £299.99
"""

In [ ]:
delimiter = "####"
context = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,\
i.e. {delimiter}.

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific product or products. \
Product cateogry doesn't count.

Step 2:{delimiter} If the user is asking about \
specific products, identify whether \
the products are in the following list.
All available products:
{products}

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.
"""

In [ ]:
chat = chat_model.start_chat(
    context=context,
    examples=[]
)

user_message = f"""
How much more expensive is the BrewBlend Pro vs the Caffeino Classic?
"""
response = chat.send_message(user_message, **parameters)
print(response.text)

The delimiters can help select different parts of the responses. We first, however, have to convert the object returned by the chat into a string.

In [ ]:
# Vertex returns a TextGenerationResponse
type(response)

In [ ]:
final_response = str(response)
print(final_response)

In [ ]:
try:
    final_response = str(response).split(delimiter)[-1].strip()
except Exception as e:
    final_response = "Sorry, I'm unsure of the answer, please try asking another."

print(final_response)